In [1]:
# 인덱싱 전략
# 1. 청킹은.. 하지 말까? 할까? 의미 단위 청킹?
# => 정책의 범위 바깥에서 청킹된 각 정보는 유의미할까? 정책이라는 틀이 없이 지원 조건, 정책 설명 내용 등이 유의미한가?
# 2. page_content는 벡터 서칭용으로만 사용하고, context로는 metadata 섹션을 활용하여 원문을 전달한다.
# 3. 이때, 원문 중 필요한 정보만 선별하여 metadata에 적재한다.



In [2]:
# 1. 벡터 서칭용 컬럼을 추가한 csv 로드.
import pandas as pd

data = pd.read_csv('./policies_with_documents.csv')
data.head()

,plcyNo,bscPlanCycl,bscPlanPlcyWayNo,bscPlanFcsAsmtNo,bscPlanAsmtNo,pvsnInstGroupCd,plcyPvsnMthdCd,plcyAprvSttsCd,plcyNm,plcyKywdNm,...,rgtrHghrkInstCdNm,zipCd,plcyMajorCd,jobCd,schoolCd,aplyYmd,frstRegDt,lastMdfcnDt,sbizCd,document
0,20250717005400211359,1,4,16,41,54002,42002.0,44002,웹툰캠퍼스 운영 및 인력 양성,"교육지원,인턴,맞춤형상담서비스,벤처",...,대구광역시,"27110,27140,27170,27200,27230,27260,27290,2771...",0011009,0013010,0049010,20250501 ~ 20250530,2025-07-17 16:00:49,2025-07-17 17:18:02,0014010,정책명은 '웹툰캠퍼스 운영 및 인력 양성'입니다. 정책 분야는 '복지문화 > 예술인...
1,20250717005400211358,1,4,16,43,54002,42004.0,44002,학예사 인턴 운영,인턴,...,전북특별자치도,"11110,11140,11170,11200,11215,11230,11260,1129...",0011006,0013003,"0049007,0049008,0049009",NaN,2025-07-17 14:38:21,2025-07-17 14:56:21,"0014008,0014009",정책명은 '학예사 인턴 운영'입니다. 정책 분야는 '복지문화 > 예술인지원'입니다....
2,20250717005400211357,1,4,16,43,54002,42004.0,44002,전북청년 2025 (전북청년 미술 운영),보조금,...,전북특별자치도,"52111,52113,52130,52140,52180,52190,52210,5271...",0011009,0013010,0049010,20241007 ~ 20241018,2025-07-17 14:32:05,2025-07-17 15:56:14,0014010,정책명은 '전북청년 2025 (전북청년 미술 운영)'입니다. 정책 분야는 '복지문화...
3,20250717005400211356,1,1,3,6,54002,42013.0,44002,제주특별자치도 공공기관 통합채용,장기미취업청년,...,제주특별자치도,"50110,50130",0011009,0013010,0049010,NaN,2025-07-17 14:29:43,2025-07-17 14:52:34,0014010,정책명은 '제주특별자치도 공공기관 통합채용'입니다. 정책 분야는 '일자리 > 취업'...
4,20250717005400211355,1,5,18,47,54002,42013.0,44002,제3차 대구광역시 청년정책 기본계획 수립,교육지원,...,대구광역시,"27110,27140,27170,27200,27230,27260,27290,2771...",0011009,0013010,0049010,NaN,2025-07-17 13:57:33,2025-07-17 14:23:02,0014010,정책명은 '제3차 대구광역시 청년정책 기본계획 수립'입니다. 정책 분야는 '참여권리...


In [4]:
# 2. metadata로 적재할 컬럼 지정
# 온통 청년 API 제공 목록 페이지를 참고.

In [30]:
# 도큐먼트로 만들기

import csv
from langchain_core.documents import Document

def create_documents_from_csv(csv_file_path: str) -> list[Document]:
    """
    CSV 파일에서 정책 정보를 읽어와
    LangChain의 Document 객체 리스트로 변환합니다.

    - 각 row는 하나의 정책 데이터를 나타냅니다.
    - 서술형 정보는 page_content로 조합합니다.
    - 정형 정보는 metadata로 저장합니다.
    """
    
    documents = []
    
    try:
        # UTF-8 인코딩으로 CSV 파일을 엽니다.
        with open(csv_file_path, mode='r', encoding='utf-8-sig') as csvfile:
            # 각 row를 딕셔너리 형태로 읽어옵니다.
            reader = csv.DictReader(csvfile)
            
            for row in reader:
                # 1. page_content 구성: 검색의 대상이 될 자연어 텍스트
                page_content = row.get('document')

                # 2. metadata 구성: 필터링 및 출처 표시에 사용할 정형 데이터
                # 항상 원본값 그대로
                metadata = {
                    # 정책 기본 정보
                    "plcyNo": row.get('plcyNo', '정보 없음'),
                    "plcyNm": row.get('plcyNm', '정보 없음'),
                    "plcyKywdNm": row.get('plcyKywdNm', '정보 없음'),
                    "plcyExplnCn": row.get('plcyExplnCn', '정보 없음'),
                    "lclsfNm": row.get('lclsfNm', '정보 없음'),
                    "mclsfNm": row.get('mclsfNm', '정보 없음'),
                    "plcySprtCn": row.get('plcySprtCn', '정보 없음'),
                    "plcyPvsnMthdCd": row.get('plcyPvsnMthdCd', '정보 없음'),

                    # 기관 정보
                    "sprvsnInstCdNm": row.get('sprvsnInstCdNm', '정보 없음'),
                    "operInstCdNm": row.get('operInstCdNm', '정보 없음'),

                    # 기간 정보
                    "aplyPrdSeCd": row.get('aplyPrdSeCd', '정보 없음'),
                    "bizPrdSeCd": row.get('bizPrdSeCd', '정보 없음'),
                    "bizPrdBgngYmd": row.get('bizPrdBgngYmd', '정보 없음'),
                    "bizPrdEndYmd": row.get('bizPrdEndYmd', '정보 없음'),
                    "bizPrdEtcCn": row.get('bizPrdEtcCn', '정보 없음'),
                    "aplyYmd": row.get('aplyYmd', '정보 없음'),
                    "frstRegDt": row.get('frstRegDt', '정보 없음'),
                    "lastMdfcnDt": row.get('lastMdfcnDt', '정보 없음'),

                    # 신청 및 방법
                    "plcyAplyMthdCn": row.get('plcyAplyMthdCn', '정보 없음'),
                    "srngMthdCn": row.get('srngMthdCn', '정보 없음'),
                    "sbmsnDcmntCn": row.get('sbmsnDcmntCn', '정보 없음'),
                    "aplyUrlAddr": row.get('aplyUrlAddr', '정보 없음'),

                    # 지원 조건
                    "sprtSclLmtYn": row.get('sprtSclLmtYn', '정보 없음'),
                    "sprtTrgtMinAge": row.get('sprtTrgtMinAge', '정보 없음'),
                    "sprtTrgtMaxAge": row.get('sprtTrgtMaxAge', '정보 없음'),
                    "sprtTrgtAgeLmtYn": row.get('sprtTrgtAgeLmtYn', '정보 없음'),
                    "mrgSttsCd": row.get('mrgSttsCd', '정보 없음'),
                    "earnMinAmt": row.get('earnMinAmt', '정보 없음'),
                    "earnMaxAmt": row.get('earnMaxAmt', '정보 없음'),
                    "earnEtcCn": row.get('earnEtcCn', '정보 없음'),
                    "addAplyQlfcCndCn": row.get('addAplyQlfcCndCn', '정보 없음'),
                    "ptcpPrpTrgtCn": row.get('ptcpPrpTrgtCn', '정보 없음'),

                    # 요건 코드(target, 대상)
                    "zipCd": row.get('zipCd', '정보 없음'),
                    "plcyMajorCd": row.get('plcyMajorCd', '정보 없음'),
                    "jobCd": row.get('jobCd', '정보 없음'),
                    "schoolCd": row.get('schoolCd', '정보 없음'),
                    "sbizCd": row.get('sbizCd', '정보 없음'),

                    # 기타
                    "etcMttrCn": row.get('etcMttrCn', '정보 없음'),
                    "refUrlAddr1": row.get('refUrlAddr1', '정보 없음'),
                    "refUrlAddr2": row.get('refUrlAddr2', '정보 없음'),
                    
                    # 필
                }

                
                documents.append(Document(page_content=page_content.strip(), metadata=metadata))
                
    except FileNotFoundError:
        print(f"오류: '{csv_file_path}' 파일을 찾을 수 없습니다.")
    except Exception as e:
        print(f"파일을 읽는 중 오류가 발생했습니다: {e}")
        
    return documents


In [33]:
docs = create_documents_from_csv('./policies_with_documents.csv')

In [34]:
def summ_docs_info(docs: list, num: int=5):
    print(f'document의 총 개수 : {len(docs)}')
    for i in docs[:num]:
        print(f"---Document ---")
        print("Page Content:")
        print(i.page_content)
        print("\nMetadata:")
        print(i.metadata)

summ_docs_info(docs)

document의 총 개수 : 3861
---Document ---
Page Content:
정책명은 '웹툰캠퍼스 운영 및 인력 양성'입니다. 정책 분야는 '복지문화 > 예술인지원'입니다. 주요 키워드는 '교육지원, 인턴, 맞춤형상담서비스, 벤처'입니다. 정책 설명: - 현장 실무형 전문인력 프로그램을 통한 글로벌 웹툰작가 양성
- 웹툰 창작 및 제작 역량 강화, 산업 활성화 지원 지원 내용: - 지역 웹툰작가 양성을 위한 교육 및 프로그램 운영
- 교육생 월 창작지원금 및 원고료 지원 등

Metadata:
{'plcyNo': '20250717005400211359', 'plcyNm': '웹툰캠퍼스 운영 및 인력 양성', 'plcyKywdNm': '교육지원,인턴,맞춤형상담서비스,벤처', 'plcyExplnCn': '- 현장 실무형 전문인력 프로그램을 통한 글로벌 웹툰작가 양성\n- 웹툰 창작 및 제작 역량 강화, 산업 활성화 지원', 'lclsfNm': '복지문화', 'mclsfNm': '예술인지원', 'plcySprtCn': '- 지역 웹툰작가 양성을 위한 교육 및 프로그램 운영\n- 교육생 월 창작지원금 및 원고료 지원 등', 'plcyPvsnMthdCd': '42002.0', 'sprvsnInstCdNm': '문화콘텐츠과', 'operInstCdNm': '재단법인대구디지털혁신진흥원', 'aplyPrdSeCd': '57001', 'bizPrdSeCd': '56001.0', 'bizPrdBgngYmd': '20250501', 'bizPrdEndYmd': '20251231', 'bizPrdEtcCn': '', 'aplyYmd': '20250501 ~ 20250530', 'frstRegDt': '2025-07-17 16:00:49', 'lastMdfcnDt': '2025-07-17 17:18:02', 'plcyAplyMthdCn': '', 'srngMthdCn': '', 'sbmsnDcmntCn': '', 'aplyUrlAddr': '', 'sprtSclLmt

In [35]:
docs[:3]

[Document(metadata={'plcyNo': '20250717005400211359', 'plcyNm': '웹툰캠퍼스 운영 및 인력 양성', 'plcyKywdNm': '교육지원,인턴,맞춤형상담서비스,벤처', 'plcyExplnCn': '- 현장 실무형 전문인력 프로그램을 통한 글로벌 웹툰작가 양성\n- 웹툰 창작 및 제작 역량 강화, 산업 활성화 지원', 'lclsfNm': '복지문화', 'mclsfNm': '예술인지원', 'plcySprtCn': '- 지역 웹툰작가 양성을 위한 교육 및 프로그램 운영\n- 교육생 월 창작지원금 및 원고료 지원 등', 'plcyPvsnMthdCd': '42002.0', 'sprvsnInstCdNm': '문화콘텐츠과', 'operInstCdNm': '재단법인대구디지털혁신진흥원', 'aplyPrdSeCd': '57001', 'bizPrdSeCd': '56001.0', 'bizPrdBgngYmd': '20250501', 'bizPrdEndYmd': '20251231', 'bizPrdEtcCn': '', 'aplyYmd': '20250501 ~ 20250530', 'frstRegDt': '2025-07-17 16:00:49', 'lastMdfcnDt': '2025-07-17 17:18:02', 'plcyAplyMthdCn': '', 'srngMthdCn': '', 'sbmsnDcmntCn': '', 'aplyUrlAddr': '', 'sprtSclLmtYn': 'N', 'sprtTrgtMinAge': '0.0', 'sprtTrgtMaxAge': '0.0', 'sprtTrgtAgeLmtYn': 'Y', 'mrgSttsCd': '55003.0', 'earnMinAmt': '0.0', 'earnMaxAmt': '0.0', 'earnEtcCn': '', 'addAplyQlfcCndCn': '', 'ptcpPrpTrgtCn': '', 'zipCd': '27110,27140,27170,27200,27230,27260,27290,2771

In [18]:
# 청킹은 하지 않음. 

In [25]:
import pandas as pd
d = pd.read_csv('./policies_with_documents.csv')
print(d['plcyNo'])

0       20250717005400211359
1       20250717005400211358
2       20250717005400211357
3       20250717005400211356
4       20250717005400211355
                ...         
3856    20240104005400100001
3857    20240103005400200001
3858    20240129005400200003
3859    20240118005400200017
3860    20231102005400200007
Name: plcyNo, Length: 3861, dtype: object


In [39]:
# 임베딩
import os 
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
from langchain_chroma import Chroma
load_dotenv()

# 임베딩 모델 및 Chroma DB 준비
# embedding_model = OpenAIEmbeddings(model="text-embedding-3-small", openai_api_key=os.getenv("OPENAI_API_KEY"))
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large", openai_api_key=os.getenv("OPENAI_API_KEY"))

# Chroma DB 준비
vectorstore = Chroma(
    collection_name="policy_collection_summary_added_openai_large",
    embedding_function=embedding_model,
    persist_directory="./chroma_db_policy"
)


True

In [41]:
# 데이터 처리
def add_to_chroma_in_batches(docs: list[Document], batch_size: int = 100):
    """문서 리스트를 배치로 나누어 ChromaDB에 추가합니다."""
    
    # 전체 문서 리스트를 batch_size만큼 건너뛰며 반복
    for i in range(0, len(docs), batch_size):
        # 현재 처리할 배치 슬라이싱
        batch = docs[i:i + batch_size]
        
        # 현재 배치만 DB에 추가
        vectorstore.add_documents(documents=batch)
        
        # 진행 상황 출력
        print(f"Batch {i//batch_size + 1}/{(len(docs) - 1)//batch_size + 1} 처리 완료 ({len(batch)}개 문서 추가)")



In [42]:

# 함수 호출로 배치 처리 실행
add_to_chroma_in_batches(docs, batch_size=200) # 배치 사이즈는 조절 가능

print("\n모든 문서의 배치 처리가 완료되었습니다.")


Batch 1/20 처리 완료 (200개 문서 추가)
Batch 2/20 처리 완료 (200개 문서 추가)
Batch 3/20 처리 완료 (200개 문서 추가)
Batch 4/20 처리 완료 (200개 문서 추가)
Batch 5/20 처리 완료 (200개 문서 추가)
Batch 6/20 처리 완료 (200개 문서 추가)
Batch 7/20 처리 완료 (200개 문서 추가)
Batch 8/20 처리 완료 (200개 문서 추가)
Batch 9/20 처리 완료 (200개 문서 추가)
Batch 10/20 처리 완료 (200개 문서 추가)
Batch 11/20 처리 완료 (200개 문서 추가)
Batch 12/20 처리 완료 (200개 문서 추가)
Batch 13/20 처리 완료 (200개 문서 추가)
Batch 14/20 처리 완료 (200개 문서 추가)
Batch 15/20 처리 완료 (200개 문서 추가)
Batch 16/20 처리 완료 (200개 문서 추가)
Batch 17/20 처리 완료 (200개 문서 추가)
Batch 18/20 처리 완료 (200개 문서 추가)
Batch 19/20 처리 완료 (200개 문서 추가)
Batch 20/20 처리 완료 (61개 문서 추가)

모든 문서의 배치 처리가 완료되었습니다.


In [43]:
import tiktoken
from langchain_core.documents import Document

def count_tokens(text, model="text-embedding-3-large"):
    # OpenAI 임베딩 모델들은 cl100k_base 인코딩 사용
    encoding = tiktoken.get_encoding("cl100k_base")
    return len(encoding.encode(text))


avg = 0;
for i in docs:
    token_count = count_tokens(i.page_content)
    avg += token_count
    if token_count >= 1000:
        print(f"토큰 수: {token_count}")
print(f"총 토큰 수: {avg}")



토큰 수: 1511
토큰 수: 1054
토큰 수: 2017
토큰 수: 1170
토큰 수: 1316
토큰 수: 1208
토큰 수: 1250
토큰 수: 1057
토큰 수: 1549
토큰 수: 1370
토큰 수: 1299
토큰 수: 1517
토큰 수: 1328
토큰 수: 1274
토큰 수: 1092
토큰 수: 1265
토큰 수: 1118
토큰 수: 1209
토큰 수: 1016
토큰 수: 1209
토큰 수: 1175
토큰 수: 1260
토큰 수: 1246
토큰 수: 1392
토큰 수: 1661
토큰 수: 1718
토큰 수: 1531
토큰 수: 1026
토큰 수: 1081
토큰 수: 1195
토큰 수: 1163
토큰 수: 1063
토큰 수: 1002
토큰 수: 1625
토큰 수: 1405
토큰 수: 1122
토큰 수: 1036
토큰 수: 1253
토큰 수: 1114
토큰 수: 1200
토큰 수: 1013
토큰 수: 1001
토큰 수: 1612
토큰 수: 1063
토큰 수: 1058
토큰 수: 1759
토큰 수: 1244
토큰 수: 1204
토큰 수: 1377
토큰 수: 1071
토큰 수: 1167
토큰 수: 1522
토큰 수: 1042
토큰 수: 1545
토큰 수: 1083
토큰 수: 1054
토큰 수: 1085
토큰 수: 1751
토큰 수: 1108
토큰 수: 1061
토큰 수: 1273
토큰 수: 1246
토큰 수: 1040
토큰 수: 1266
토큰 수: 1178
토큰 수: 1050
토큰 수: 1088
토큰 수: 1058
토큰 수: 1411
토큰 수: 2389
토큰 수: 1041
토큰 수: 1340
토큰 수: 1144
토큰 수: 1102
토큰 수: 1051
토큰 수: 1049
토큰 수: 2112
총 토큰 수: 1444000
